In [2]:
import sys
import pathlib

import numpy
import scipy

import networkx

sys.path.append("../scripts")
from utils import parse_interactome, parse_causal_genes, scores_to_TSV

In [3]:
interactome, genes = parse_interactome(interactome_file="/home/kubicaj/calc/input/Interactome_human.sif")

In [4]:
causal_genes = parse_causal_genes(causal_genes_file="/home/kubicaj/calc/input/causalGenes_allPhenotypes.csv", canonical_genes_file="/home/kubicaj/calc/input/canonicalGenes.tsv", genes=genes)

In [5]:
causal_genes_array = numpy.array([1 if causal_genes.get(n) == 1 else 0 for n in interactome.nodes()])

In [6]:
A = networkx.to_scipy_sparse_array(interactome, format='csc')

In [7]:
A = A / A.sum(axis=0)

In [8]:
I = scipy.sparse.eye_array(m=A.shape[0])

In [9]:
alpha = 0.1

In [18]:
L = scipy.sparse.linalg.inv((I - alpha*A)) - I

In [19]:
scipy.sparse.save_npz("Leontief_interactome_alpha01.npz", L)

In [12]:
L = scipy.sparse.load_npz("/home/kubicaj/calc/grexome-TIMC-Secondary-interactome/Interactome/scratch/Leontief_interactome_alpha01.npz")

In [13]:
L_norm = scipy.sparse.linalg.inv((I - (alpha / 100)*A)) - I

/home/kubicaj/python_venvs/pyEnv_PPI/lib64/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:602: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
/home/kubicaj/python_venvs/pyEnv_PPI/lib64/python3.9/site-packages/scipy/sparse/linalg/_matfuncs.py:76: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  Ainv = spsolve(A, I)


In [14]:
scipy.sparse.save_npz("Leontief_interactome_norm_alpha01_div100.npz", L)

In [21]:
scores_array = numpy.dot(L.todense(), causal_genes_array)

In [22]:
ones_arr = numpy.ones(shape=L_norm.shape[0])

In [23]:
norm_array = numpy.dot(L_norm.todense(), ones_arr)

In [24]:
scores_array_normalized = numpy.squeeze(scores_array / norm_array)

In [25]:
scores = dict(zip(interactome.nodes(), scores_array_normalized))

In [26]:
scores_to_TSV(scores, out_path=pathlib.Path('/home/kubicaj/calc/grexome-TIMC-Secondary-interactome/Interactome/scratch'), file_name=pathlib.Path('scores_Leontief_alpha01_v3.tsv'))